In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iamthebestcoderopen2020/purchase_detail.csv
/kaggle/input/iamthebestcoderopen2020/user_label_train.csv
/kaggle/input/iamthebestcoderopen2020/submission.csv
/kaggle/input/iamthebestcoderopen2020/user_info.csv
/kaggle/input/iamthebestcoderopen2020/login.csv


In [2]:
import pandas as pd

In [3]:
df_purchase_detail = pd.read_csv('/kaggle/input/iamthebestcoderopen2020/purchase_detail.csv')
df_user_label_train = pd.read_csv('/kaggle/input/iamthebestcoderopen2020/user_label_train.csv')
df_user_info = pd.read_csv('/kaggle/input/iamthebestcoderopen2020/user_info.csv')
df_login = pd.read_csv('/kaggle/input/iamthebestcoderopen2020/login.csv')
submit = pd.read_csv('/kaggle/input/iamthebestcoderopen2020/submission.csv')

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

for _df in [df_purchase_detail,df_user_label_train,df_user_info,df_login]:
    _df = reduce_mem_usage(_df,verbose=False)

In [5]:
log_fix = df_login.groupby('userid').agg({
    'login_times':['mean','sum','std','max'],
    'date':['count']
}).reset_index()

In [6]:
log_fix.columns = [x[0]+x[1] for x in log_fix.columns]

In [7]:
import datetime

In [8]:
df_user_info.enroll_time = df_user_info.enroll_time.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
tartime = datetime.datetime.strptime('2020-08-31', '%Y-%m-%d')
df_user_info['diff_enroll'] = df_user_info.enroll_time.apply(lambda x: tartime-x)
df_user_info['birth_year'] = df_user_info['birth_year'].apply(lambda x: 2020-x)
df_user_info['diff_enroll'] = df_user_info['diff_enroll'].apply(lambda x: x.days)

In [9]:
df_user_info

,userid,gender,is_seller,birth_year,enroll_time,diff_enroll
0,1,2.0,1,35.0,2015-05-27,1923
1,2,1.0,1,40.0,2015-05-27,1923
2,3,2.0,1,6.0,2015-05-27,1923
3,4,1.0,1,42.0,2015-05-27,1923
4,5,1.0,1,42.0,2015-05-27,1923
...,...,...,...,...,...,...
502152,521605,2.0,0,16.0,2020-02-01,212
502153,521606,2.0,0,30.0,2020-02-01,212
502154,521607,1.0,0,NaN,2020-02-01,212
502155,521608,2.0,0,15.0,2020-02-01,212


In [10]:
df_user = df_user_info.merge(log_fix)
df_user = df_user.merge(df_user_label_train,how='outer')
df_user

,userid,gender,is_seller,birth_year,enroll_time,diff_enroll,login_timesmean,login_timessum,login_timesstd,login_timesmax,datecount,label
0,1,2.0,1,35.0,2015-05-27,1923,1.388889,50.0,0.687761,4,36,0.0
1,2,1.0,1,40.0,2015-05-27,1923,2.675824,487.0,1.179647,6,182,0.0
2,3,2.0,1,6.0,2015-05-27,1923,4.230769,770.0,1.212977,8,182,0.0
3,4,1.0,1,42.0,2015-05-27,1923,2.393258,426.0,0.998507,5,178,0.0
4,5,1.0,1,42.0,2015-05-27,1923,4.527473,824.0,1.336613,10,182,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
502152,521605,2.0,0,16.0,2020-02-01,212,1.312500,63.0,0.776140,4,48,NaN
502153,521606,2.0,0,30.0,2020-02-01,212,1.102273,97.0,0.304743,2,88,0.0
502154,521607,1.0,0,NaN,2020-02-01,212,1.233129,201.0,0.465747,3,163,0.0
502155,521608,2.0,0,15.0,2020-02-01,212,1.107843,113.0,0.311714,2,102,0.0


In [11]:
cat_map = df_purchase_detail['category_encoded'].value_counts(normalize=True)

In [12]:
df_purchase_detail['cat_encode'] = df_purchase_detail['category_encoded'].map(cat_map)

In [13]:
df_purchase_detail['epochtime'] = pd.to_datetime(df_purchase_detail['grass_date']).apply(lambda x :x.strftime('%s'))

In [17]:
df_purchase_detail['epochtime'] = df_purchase_detail['epochtime'].astype('int32')

In [18]:
xx = df_purchase_detail.groupby('userid').agg({
    'order_count':['count','mean','std','median','sum'],
    'total_amount':['count','mean','std','median','sum'],
    'cat_encode':['sum','mean','std','max','min'],
    'epochtime':['std','max','mean']
})
xx = xx.reset_index()
xx

userid order_count                                total_amount  \
                     count      mean       std median sum        count   
0            1          10  3.000000  2.054805    3.0  30           10   
1            2          18  1.388889  0.607685    1.0  25           18   
2            3           7  2.000000  1.000000    2.0  14            7   
3            4          35  1.542857  1.196634    1.0  54           35   
4            5          65  1.323077  0.772732    1.0  86           65   
...        ...         ...       ...       ...    ...  ..          ...   
502152  521605           6  2.500000  3.209361    1.0  15            6   
502153  521606          20  3.300000  4.921489    1.0  66           20   
502154  521607          17  3.176471  3.609628    2.0  54           17   
502155  521608           4  3.250000  2.217356    3.0  13            4   
502156  521609          18  1.611111  1.243283    1.0  29           18   

                                           cat_encode                      \
             mean         std median   sum        sum      mean       std   
0       24.600000   44.962206   10.0   246   0.845411  0.084541  0.058914   
1        3.277778    2.926664    1.5    59   2.345530  0.130307  0.077181   
2        2.000000    1.000000    2.0    14   0.830990  0.118713  0.084753   
3        2.342857    2.235316    1.0    82   2.760878  0.078882  0.076074   
4       42.061538  245.887416    1.0  2734   4.309073  0.066293  0.060583   
...           ...         ...    ...   ...        ...       ...       ...   
502152   2.666667    3.141125    1.5    16   0.537552  0.089592  0.050298   
502153   7.000000   12.156869    2.5   140   1.589052  0.079453  0.055950   
502154   3.411765    3.641065    2.0    58   1.089842  0.064108  0.057997   
502155   4.250000    2.362908    5.0    17   0.661482  0.165370  0.069160   
502156   2.055556    2.127474    1.0    37   1.507726  0.083763  0.075946   

                               epochtime                            
             max       min           std         max          mean  
0       0.199950  0.037988  5.676742e+06  1595980800  1.590019e+09  
1       0.199950  0.029049  4.334120e+06  1595894400  1.589554e+09  
2       0.199950  0.004466  2.882385e+06  1592006400  1.589118e+09  
3       0.199950  0.007514  4.925288e+06  1595203200  1.587457e+09  
4       0.199950  0.007514  4.761194e+06  1595548800  1.588577e+09  
...          ...       ...           ...         ...           ...  
502152  0.131296  0.009469  2.212526e+06  1587081600  1.583035e+09  
502153  0.199950  0.031166  3.600911e+06  1594080000  1.586183e+09  
502154  0.199950  0.007514  5.272342e+06  1595548800  1.589298e+09  
502155  0.199950  0.061631  3.887680e+06  1590192000  1.586304e+09  
502156  0.199950  0.009469  4.529511e+06  1595808000  1.588152e+09  

[502157 rows x 19 columns]

In [19]:
xx.columns = [x[0]+x[1] for x in xx.columns]
xx = reduce_mem_usage(xx)

Mem. usage decreased to 22.99 Mb (65.2% reduction)


In [20]:
df_user = reduce_mem_usage(df_user)

Mem. usage decreased to 18.20 Mb (52.5% reduction)


In [21]:
df_user = df_user.merge(xx,how='left')

In [22]:
df_user

,userid,gender,is_seller,birth_year,enroll_time,diff_enroll,login_timesmean,login_timessum,login_timesstd,login_timesmax,...,total_amountmedian,total_amountsum,cat_encodesum,cat_encodemean,cat_encodestd,cat_encodemax,cat_encodemin,epochtimestd,epochtimemax,epochtimemean
0,1,2.0,1,35.0,2015-05-27,1923,1.388672,50.0,0.687988,4,...,10.0,246,0.845215,0.084534,0.058929,0.199951,0.037994,5676742.50,1595980800,1.590019e+09
1,2,1.0,1,40.0,2015-05-27,1923,2.675781,487.0,1.179688,6,...,1.5,59,2.345703,0.130249,0.077209,0.199951,0.029053,4334120.00,1595894400,1.589554e+09
2,3,2.0,1,6.0,2015-05-27,1923,4.230469,770.0,1.212891,8,...,2.0,14,0.831055,0.118713,0.084778,0.199951,0.004467,2882385.25,1592006400,1.589118e+09
3,4,1.0,1,42.0,2015-05-27,1923,2.392578,426.0,0.998535,5,...,1.0,82,2.761719,0.078857,0.076050,0.199951,0.007515,4925288.00,1595203200,1.587457e+09
4,5,1.0,1,42.0,2015-05-27,1923,4.527344,824.0,1.336914,10,...,1.0,2734,4.308594,0.066284,0.060577,0.199951,0.007515,4761194.00,1595548800,1.588577e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502152,521605,2.0,0,16.0,2020-02-01,212,1.312500,63.0,0.776367,4,...,1.5,16,0.537598,0.089600,0.050293,0.131348,0.009468,2212526.25,1587081600,1.583035e+09
502153,521606,2.0,0,30.0,2020-02-01,212,1.102539,97.0,0.304688,2,...,2.5,140,1.588867,0.079468,0.055939,0.199951,0.031174,3600910.50,1594080000,1.586183e+09
502154,521607,1.0,0,NaN,2020-02-01,212,1.233398,201.0,0.465820,3,...,2.0,58,1.089844,0.064087,0.057983,0.199951,0.007515,5272342.00,1595548800,1.589298e+09
502155,521608,2.0,0,15.0,2020-02-01,212,1.107422,113.0,0.311768,2,...,5.0,17,0.661621,0.165405,0.069153,0.199951,0.061646,3887680.00,1590192000,1.586304e+09


In [23]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpsjq0d6s6
  JVM stdout: /tmp/tmpsjq0d6s6/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpsjq0d6s6/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,H2O_from_python_unknownUser_s3y4lr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [24]:
df_user.columns

Index(['userid', 'gender', 'is_seller', 'birth_year', 'enroll_time',
       'diff_enroll', 'login_timesmean', 'login_timessum', 'login_timesstd',
       'login_timesmax', 'datecount', 'label', 'order_countcount',
       'order_countmean', 'order_countstd', 'order_countmedian',
       'order_countsum', 'total_amountcount', 'total_amountmean',
       'total_amountstd', 'total_amountmedian', 'total_amountsum',
       'cat_encodesum', 'cat_encodemean', 'cat_encodestd', 'cat_encodemax',
       'cat_encodemin', 'epochtimestd', 'epochtimemax', 'epochtimemean'],
      dtype='object')

In [25]:
htrain = h2o.H2OFrame(df_user[df_user.label.notnull()])
htest = h2o.H2OFrame(df_user[df_user.label.isnull()].drop(columns='label'))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [26]:
df_user.columns

Index(['userid', 'gender', 'is_seller', 'birth_year', 'enroll_time',
       'diff_enroll', 'login_timesmean', 'login_timessum', 'login_timesstd',
       'login_timesmax', 'datecount', 'label', 'order_countcount',
       'order_countmean', 'order_countstd', 'order_countmedian',
       'order_countsum', 'total_amountcount', 'total_amountmean',
       'total_amountstd', 'total_amountmedian', 'total_amountsum',
       'cat_encodesum', 'cat_encodemean', 'cat_encodestd', 'cat_encodemax',
       'cat_encodemin', 'epochtimestd', 'epochtimemax', 'epochtimemean'],
      dtype='object')

In [ ]:
df_user

In [ ]:
x = [x for x in df_user.columns if x not in ['label']]
y = 'label'

# For binary classification, response should be a factor
htrain[y] = htrain[y].asfactor()
aml = H2OAutoML(max_models=500, 
                seed=20201023,
                max_runtime_secs=60, 
                sort_metric="auc", 
                stopping_metric="auc")

aml.train(x=x, y=y, training_frame=htrain)

AutoML progress: |███████████████████████████

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
test_y = aml.leader.predict(htest)

In [ ]:
submit1 = pd.DataFrame({
    'userid': df_user[df_user.label.isnull()]['userid'].values,
    'label':test_y.as_data_frame()['p1']
})
submit1

In [ ]:
final = submit.merge(submit1)

In [ ]:
final.to_csv('output_v5.csv',index=0)